In [3]:
# Instala e carrega pacotes
import pandas as pd

import numpy as np

In [7]:
# Coleta dados
catalogoVulnerabilidades = pd.read_csv('known_exploited_vulnerabilities.csv', header=0, sep=',')

# Criar coluna booleana state com 0 para Unknown e 1 para Known
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = np.where(catalogoVulnerabilidades['knownRansomwareCampaignUse'] == 'Known', 1, 0 )

#Reordenando as colunas
catalogoVulnerabilidades = catalogoVulnerabilidades[['cveID', 
                                                     'vendorProject',
                                                     'product',
                                                     'vulnerabilityName',
                                                     'dateAdded',
                                                     'shortDescription',
                                                     'requiredAction',
                                                     'dueDate',
                                                     'knownRansomwareCampaignUse',
                                                     'codigoKnownRansomwareCampaignUse',
                                                     'notes']]    

catalogoVulnerabilidades.head()

,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,knownRansomwareCampaignUse,codigoKnownRansomwareCampaignUse,notes
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,1,NaN
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,1,NaN
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,Known,1,NaN
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,Known,1,NaN
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,Unknown,0,NaN


In [ ]:
# Tokenização

In [ ]:
# Stop Words

In [ ]:
# Lematização